In [1]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import trange

from tensorflow.keras import mixed_precision

### Test new detector class

In [2]:
# import
from detection.model import MaskRCNN

In [3]:
mask_model = MaskRCNN('models/mask_rcnn_inception_resnet_v2/saved_model/')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [5]:
w, h = 1700, 1700
fps = 10
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
writer = cv2.VideoWriter('Movie_1_result.mp4', fourcc, fps, (w, h))

c = 0
capture = cv2.VideoCapture('Movie_1.mkv')
for i in trange(60*10 + 1):
    ret, frame = capture.read()
    if frame is None:
        break
    new_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    for frame_coords in mask_model.detect(new_frame):
    
        print(frame_coords)
        if c >= 5:
            break
        c += 1
    
#     new_frame = draw_predictions(new_frame, raw_result)
    
#     writer.write(cv2.cvtColor(new_frame, cv2.COLOR_RGB2BGR))

# writer.release() 
    
# capture.release()

  0%|          | 2/601 [00:07<32:10,  3.22s/it]  

Coords(x_min=1163, y_min=409, x_max=1396, y_max=795)
Coords(x_min=1331, y_min=620, x_max=1521, y_max=971)
Coords(x_min=252, y_min=523, x_max=519, y_max=796)
Coords(x_min=202, y_min=317, x_max=420, y_max=694)
Coords(x_min=1281, y_min=861, x_max=1511, y_max=1198)
Coords(x_min=588, y_min=680, x_max=736, y_max=1045)


  0%|          | 3/601 [00:08<19:46,  1.98s/it]

Coords(x_min=202, y_min=317, x_max=419, y_max=678)


  1%|          | 4/601 [00:08<13:57,  1.40s/it]

...

Coords(x_min=1169, y_min=407, x_max=1397, y_max=796)


  2%|▏         | 14/601 [00:13<05:10,  1.89it/s]

Coords(x_min=203, y_min=327, x_max=406, y_max=668)


  2%|▏         | 14/601 [00:14<09:52,  1.01s/it]


KeyboardInterrupt: 